the goal is to create a regression model where the model is fed a time series somehow

In [2]:
import graphlab as gl
import datetime
from datetime import timedelta


In [3]:
prices = gl.SFrame.read_csv('coinbaseUSD.csv',header=False)

This non-commercial license of GraphLab Create for academic use is assigned to Nathanclaird@gmail.com and will expire on December 14, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\ADMIN\AppData\Local\Temp\graphlab_server_1514007158.log.0


Finished parsing file C:\Users\ADMIN\notebook\Bitcoin-Market-Regression\coinbaseUSD.csv

Parsing completed. Parsed 100 lines in 0.90626 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


...

Read 9753264 lines. Lines per second: 1.31597e+006

Read 18166602 lines. Lines per second: 1.39065e+006

Read 26481207 lines. Lines per second: 1.41598e+006

Finished parsing file C:\Users\ADMIN\notebook\Bitcoin-Market-Regression\coinbaseUSD.csv

Parsing completed. Parsed 27873479 lines in 18.9863 secs.

In [4]:
prices.head()
prices['int_times']=prices['X1']

In [5]:
prices['X1']= prices['X1'].apply(lambda x: datetime.datetime.utcfromtimestamp(x) )

In [6]:
prices.rename({'X1':'date','X2':'price','X3':'volume'})

date,price,volume,int_times
2014-12-01 05:33:56,300.0,0.01,1417412036
2014-12-01 05:40:23,300.0,0.01,1417412423
2014-12-01 06:24:08,370.0,0.01,1417415048
2014-12-01 06:50:12,370.0,0.02655554,1417416612
2014-12-02 05:29:26,377.0,0.01,1417498166
2014-12-02 10:59:09,377.75,0.25,1417517949
2014-12-02 10:59:09,378.0,3.75,1417517949
2014-12-02 11:04:17,378.0,4.9,1417518257
2014-12-02 11:05:40,378.0,5.2,1417518340
2014-12-02 18:43:00,378.0,0.1,1417545780


In [7]:
prices['volume'].max()

897.982

In [8]:
time_price= gl.TimeSeries(prices,index='date')

In [9]:
delta_day = timedelta(days=1)
delta_hour =  timedelta(hours=2) 

In [10]:
daily_agg=time_price.resample(delta_day,downsample_method='mean',upsample_method='linear')
hourly_agg=time_price.resample(delta_hour,downsample_method='mean',upsample_method='linear')

In [11]:
roll_period=[1,2,3,4,5,10,20,30,40,50,100,200,300,400,500]

In [12]:
for x in roll_period:
    temp_name = 'pri_rol_'+str(x)
    daily_agg[temp_name]=daily_agg['price'].rolling_mean(-1-x,-1,min_observations=0)

In [13]:
for x in roll_period:
    temp_name = 'vol_rol_'+str(x)
    daily_agg[temp_name]=daily_agg['volume'].rolling_sum(-1-x,-1,min_observations=0)

In [14]:
for x in roll_period:
    temp_name = 'pri_rol_'+str(x)
    hourly_agg[temp_name]=hourly_agg['price'].rolling_mean(-1-x,-1,min_observations=0)

In [15]:
for x in roll_period:
    temp_name = 'vol_rol_'+str(x)
    hourly_agg[temp_name]=hourly_agg['volume'].rolling_sum(-1-x,-1,min_observations=0)

In [16]:
model_data=daily_agg.to_sframe().remove_column('date').dropna()



In [17]:
model_data.head()

int_times,price,volume,pri_rol_1,pri_rol_2,pri_rol_3,pri_rol_4
1417526708.12,377.84375,1.8767,335.0,335.0,335.0,335.0
1417612312.0,377.818333333,0.0911014033333,356.421875,356.421875,356.421875,356.421875
1417657685.0,377.1,0.01,377.831041667,363.554027778,363.554027778,363.554027778
1417743885.0,377.55,0.0125,377.459166667,377.587361111,366.940520833,366.940520833
1417830085.0,378.0,0.015,377.325,377.489444444,377.578020833,369.062416667
1417953835.5,376.5025,0.06625,377.775,377.55,377.617083333,377.662416667
1418077586.0,375.005,0.1175,377.25125,377.350833333,377.288125,377.394166667
1418128481.25,377.1275,0.06375,375.75375,376.5025,376.764375,376.8315
1418179376.5,379.25,0.01,376.06625,376.211666667,376.65875,376.837
1418267479.45,371.074,0.32,378.18875,377.1275,376.97125,377.177


In [18]:
 model_train, model_test = model_data.random_split(.75, seed=5)

In [19]:
model_date_train_mask = model_data['int_times']>model_data['int_times'].mean()
model_date_train = model_data[model_date_train_mask]

In [20]:
model_date_test_mask = model_data['int_times']<model_data['int_times'].mean()
model_date_test = model_data[model_date_test_mask]

In [21]:
evalSFrame = gl.SFrame()
evaldateSFrame = gl.SFrame()
model_date = gl.regression.create(model_date_train, target='price')
model = gl.regression.create(model_train, target='price')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Boosted trees regression:

--------------------------------------------------------

Number of examples          : 526

Number of features          : 32

...

| 3         | 0.024307     | 8083.557617        | 1240.217773          | 1034.426514   | 344.369385      |

| 4         | 0.034002     | 6236.086914        | 820.843262           | 751.943665    | 225.160873      |

| 5         | 0.039141     | 4873.578125        | 531.469238           | 549.448730    | 152.586136      |

| 6         | 0.044611     | 3868.727539        | 292.920410           | 404.547516    | 95.961372       |

+-----------+--------------+--------------------+----------------------+---------------+-----------------+

In [22]:
price_prediction = model.predict(model_test)
price_date_prediction = model_date.predict(model_date_test)

In [23]:
price_prediction.size()

277

In [24]:
evalSFrame['real'] = model_test['price']
evalSFrame['predict'] = price_prediction
evalSFrame['per_err'] = evalSFrame['real']/(evalSFrame['predict'])
evalSFrame['per_err'].mean()

1.0349749286837555

In [25]:
evaldateSFrame['real'] = model_date_test['price']
evaldateSFrame['predict'] = price_date_prediction
evaldateSFrame['per_err'] = evaldateSFrame['real']/(evaldateSFrame['predict'])
evaldateSFrame['per_err'].mean()

0.5493472359793151

In [26]:
evaldateSFrame.show(view='ScatterPlot')

Canvas is accessible via web browser at the URL: http://localhost:60639/index.html
Opening Canvas in default web browser.


In [27]:
evaldateSFrame.head()

real,predict,per_err
377.84375,594.566345215,0.635494681192
377.818333333,594.566345215,0.635451932949
377.1,594.566345215,0.634243769488
377.55,594.566345215,0.635000623629
378.0,594.566345215,0.635757477769
376.5025,594.566345215,0.63323883538
375.005,594.566345215,0.630720192991
377.1275,594.566345215,0.634290021686
379.25,594.566345215,0.63785985038
371.074,594.566345215,0.624108651602


evalSFrame['error_per'] = evalSFrame['real'].divide(evalSFrame['predict'])